In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1,3,4"
os.environ['TRANSFORMERS_CACHE'] = '/raid/rabikov/hf_cache/'
os.environ['HF_HOME'] = '/raid/rabikov/hf_cache/'

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, LlamaTokenizer, LlamaForCausalLM
import torch





/raid/rabikov/anaconda3/envs/conda_taxonomy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

model_id = "decapoda-research/llama-30b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [ ]:

tokenizer = LlamaTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(model_id,
                                            quantization_config=bnb_config,
                                            device_map="auto")

In [37]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [38]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, 
    lora_alpha=16, 
   # target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [39]:
model.print_trainable_parameters()

trainable params: 12,779,520 || all params: 16,490,646,016 || trainable%: 0.07749556923119148


In [4]:
txt = tokenizer('hello how are you doing?'D)

In [25]:
label = tokenizer.encode('i am doing well', return_tensors='pt', add_special_tokens=False)

In [26]:
label[0]

tensor([ 474,  626, 2599, 1532])

In [28]:
input_seq = torch.concat([txt['input_ids'], label], dim=1)

In [31]:
processed_term  = 'how are you?'
target = "great!"


encoded_term = tokenizer.encode(
    processed_term, return_tensors='pt'
)
encoded_target = tokenizer.encode(target, return_tensors='pt', add_special_tokens=False)

input_seq = torch.concat([encoded_term, encoded_target], dim=1)
labels = input_seq.clone()
labels[0, : encoded_term.size()[1]] = -100

att_mask_inputs = torch.zeros_like(input_seq)
att_mask_inputs[input_seq != 0] = 1

In [34]:
labels, input_seq

(tensor([[ -100,  -100,  -100,  -100,  -100,  2107, 29991]]),
 tensor([[    0,   920,   526,   366, 29973,  2107, 29991]]))

In [40]:
out = model.forward(input_ids=input_seq, attention_mask=att_mask_inputs, labels=labels)

In [42]:
loss = out.loss

In [43]:
loss.backward()

In [44]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

In [45]:
optimizer.step()

In [46]:
out = model.forward(input_ids=input_seq, attention_mask=att_mask_inputs, labels=labels)

In [48]:
out.loss.backward()

In [49]:
optimizer.zero_grad()
optimizer.step()

In [50]:
out = model.forward(input_ids=input_seq, attention_mask=att_mask_inputs, labels=labels)

In [52]:
for i in range(10):
    optimizer.zero_grad()

    out = model.forward(input_ids=input_seq, attention_mask=att_mask_inputs, labels=labels)
    loss = out.loss
    print(loss)
    loss.backward()
    optimizer.step()

tensor(5.7083, grad_fn=<ToCopyBackward0>)
tensor(5.6528, grad_fn=<ToCopyBackward0>)
tensor(5.5960, grad_fn=<ToCopyBackward0>)
tensor(5.5263, grad_fn=<ToCopyBackward0>)
tensor(5.4481, grad_fn=<ToCopyBackward0>)
tensor(5.3696, grad_fn=<ToCopyBackward0>)
tensor(5.2882, grad_fn=<ToCopyBackward0>)
tensor(5.1871, grad_fn=<ToCopyBackward0>)
tensor(5.0791, grad_fn=<ToCopyBackward0>)
tensor(4.9481, grad_fn=<ToCopyBackward0>)
